In [1]:
import os
import pandas as pd
import json
import numpy as np
#!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#from ipynb.fs.full.my_functions import factorial

In [4]:
import tweepy
import traceback

In [5]:
import spacy
import torch

#!pip install spacymoji
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer

#!pip install transformers
from transformers import AutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

In [47]:
# Get features for TF-IDF
tfidf = TfidfVectorizer()#tokenizer=spacy_tokeniser)
tfidf.fit(train_text)
train_features = tfidf.transform(train_text)
#validation_features = tfidf.transform(val_text)
test_features = tfidf.transform(test_text)

In [5]:
#print(pipe['encoder'].get_feature_names_out())

# Define categorical columns
categorical = ['verified', 'created_at']
print(f"Categorical columns are: {categorical}")

lexical = ['description', 'location', 'name', 'source', 'text', 'username']
print(f"Lexical columns are: {lexical}")

Categorical columns are: ['verified', 'created_at']
Lexical columns are: ['description', 'location', 'name', 'source', 'text', 'username']


In [9]:
df = X.copy()
for col_name in categorical:
    df[col_name] = df[col_name].astype('category')
    df[col_name] = df[col_name].cat.codes.astype('float')
    
df[categorical].head()

,verified,created_at
71363,0.0,1.0
8081,0.0,20.0
18607,0.0,23.0
76604,0.0,6.0
58726,0.0,14.0


In [10]:
df['verified'].value_counts()

0.0    205994
1.0     11868
Name: verified, dtype: int64

In [11]:
text_combined = []
for (i, row) in df.iterrows():
    combined = ' [SEP] '.join([str(row[text_col]) for text_col in lexical])
    text_combined.append(combined)
    
df.insert(0, 'all_text', text_combined)
df.head()

,all_text,text,source,created_at,lang,description,username,verified,name,location
71363,2014 Top 100 Conservative Website. #Political ...,nazi antisemites slip gop primary causing head...,dlvr.it,1.0,en,2014 Top 100 Conservative Website. #Political ...,Briefing__Room,0.0,The Briefing Room,USA
8081,nan [SEP] nan [SEP] 17thReese [SEP] Twitter We...,nfl lawyer claimed super bowl rigged found dea...,Twitter Web Client,20.0,en,NaN,17th_reese,0.0,17thReese,NaN
18607,Democratic candidate proudly representing Conn...,honor serve people stratford connecticut house...,Twitter Web Client,23.0,en,Democratic candidate proudly representing Conn...,PhilYoungCT,0.0,Re-Elect Philip Young for State Rep,"Stratford, CT"
76604,"🍀🍀Music Film TV Sports History Politics, or wh...",transcript first debate following transcript p...,Hootsuite,6.0,en,"🍀🍀Music Film TV Sports History Politics, or wh...",davemasters,0.0,Dave Masters,🌍
58726,"Paint, coatings, color matcher, decorative, an...",412 michigan muslim arrested largest bust u hi...,Twitter Web Client,14.0,en,"Paint, coatings, color matcher, decorative, an...",jpg3211,0.0,Jpg3211,"WPB, FL"


In [188]:
#pickle.dump(tfidf, open("tfidf_vectorizer.sav", "wb"))
#print(train_text)
# Fit pipeline to training data

# https://towardsdatascience.com/pipeline-columntransformer-and-featureunion-explained-f5491f815f
# Interesting read for time-related feature engineering: 
#   https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html

# class TextProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.text_transformer = Pipeline([
#             ('tfidf_vectoriser', TfidfVectorizer()),
#         ])
       
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return self.text_transformer.fit_transform(X.squeeze()).toarray()

# class CategoryProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.text_transformer = Pipeline([
#             ('1hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         ])
       
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return self.text_transformer.fit_transform(X.squeeze()).toarray()

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    
def get_transformers(keys):
    transformers = []
    for key in keys:
        transformers.append(
            (key, Pipeline([
                ('selector', ItemSelector(key=key)),
                ('tfidf', TfidfVectorizer())
            ]))
        )
    return transformers
    
cat_pipe = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

lex_pipe = Pipeline([
    #("squeeze", FunctionTransformer(lambda x: x.squeeze())),
    ('encoder', TfidfVectorizer()),
    #("toarray", FunctionTransformer(lambda x: x.toarray()))
])

lex_pipe = FeatureUnion([
        ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf', TfidfVectorizer())])),
#         ('desc', Pipeline([
#             ('selector', ItemSelector(key='description')),
#             ('tfidf', TfidfVectorizer())])),
#         ('loc', Pipeline([
#             ('selector', ItemSelector(key='location')),
#             ('tfidf', TfidfVectorizer())])),
#         ('name', Pipeline([
#             ('selector', ItemSelector(key='name')),
#             ('tfidf', TfidfVectorizer())])),
#         ('source', Pipeline([
#             ('selector', ItemSelector(key='source')),
#             ('tfidf', TfidfVectorizer())])),
#         ('user', Pipeline([
#             ('selector', ItemSelector(key='username')),
#             ('tfidf', TfidfVectorizer())])),
])

preprocessor = ColumnTransformer([
    ('lex', lex_pipe, lexical),
    ('cat', cat_pipe, categorical),
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(warm_start=True))
    #('model', SGDClassifier(loss='log_loss'))
])

print(X_train.shape)

mini_model = SGDClassifier(loss='log_loss')

batch_size = 1000
for batch_ix in range(0, len(X_train), batch_size):
    ix = min(batch_ix + batch_size, len(X_train))
    #pipe.fit(X_train.iloc[batch_ix:ix, :], y_train.iloc[batch_ix:ix])
    mini_model.partial_fit(X_train.iloc[batch_ix:ix, :], y_train.iloc[batch_ix:ix])

#pipe.fit(X_train, y_train) # hahaha, naive thinking this would work

# Predict test data
#y_test_pred = pipe.predict(X_test)
y_test_pred = mini_model.predict(X_test)


evaluation_summary("LR multi-feature test", y_test, t_test_pred)

(98882, 9)


ValueError: could not convert string to float: 'nazi antisemites slip gop primary causing headache party httpstco5t9k5caa6q'

In [112]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [38]:
shorter = train_features[:50, :50]
shorter_labels = train_labels[:50]
print(train_features[100, 666])
#print(train_features)
print(train_features.shape)
print(shorter.shape)

# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(shorter, shorter_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(shorter, shorter_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(shorter, shorter_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

0.0
(66259, 111786)
(50, 50)


C:\Users\karli\AppData\Local\Temp\ipykernel_4036\3339626443.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  shorter_labels = train_labels[:50]


ValueError: The number of classes has to be greater than one; got 1 class

In [49]:
# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(train_features, train_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(train_features, train_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(train_features, train_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

Evaluation for: SVC test
              precision    recall  f1-score   support

       False      0.816     0.285     0.422     74981
        True      0.572     0.937     0.711     76622

    accuracy                          0.614    151603
   macro avg      0.694     0.611     0.566    151603
weighted avg      0.693     0.614     0.568    151603


Confusion matrix:
 [[21340 53641]
 [ 4824 71798]]
Evaluation for: LR (TF-IDF) test
              precision    recall  f1-score   support

       False      0.823     0.450     0.582     74981
        True      0.627     0.905     0.741     76622

    accuracy                          0.680    151603
   macro avg      0.725     0.678     0.662    151603
weighted avg      0.724     0.680     0.662    151603


Confusion matrix:
 [[33752 41229]
 [ 7256 69366]]
Evaluation for: Dummy MF test
              precision    recall  f1-score   support

       False      0.495     1.000     0.662     74981
        True      0.000     0.000     0.000    

In [ ]:
# print(train_features)

In [50]:
# Save model
import pickle
pickle.dump(lr_model_tfidf, open("simple_model.sav", "wb"))

## Make Prediction

In [ ]:
# Get output

input_text = "Watch video showing gunfire inside Canada's pa"
input_df = pd.DataFrame({"src_text": [input_text]})
input_features = tfidf.transform(input_df)

predicted_label = lr_model_tfidf.predict(input_features)

print(predicted_label)

['0']


# Import, Fix and Clean Data

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

#remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']
remove_col = ['Unnamed: 0.1', 'Unnamed: 0', 'id', 'author_id', 'edit_history_tweet_ids', 'article_id', 'Unnamed: 0.2']

# Impute null values with None
def null_process(feature_df):
    col = 'text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Combine dataset classes
real_df = pd.read_csv("data/FakeNewsNet/dataset/pf_real.csv")
fake_df = pd.read_csv("data/FakeNewsNet/dataset/pf_fake.csv")
fake_df['true'] = False
real_df['true'] = True
df = pd.concat([fake_df, real_df])

# Fix imbalance
# https://elitedatascience.com/imbalanced-classes
# Separate majority and minority classes
df_majority = df[df.true==True]
df_minority = df[df.true==False]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

df_downsampled = df_downsampled.sample(frac=1, random_state=111019)

# Split train-test
# https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group

# Update df to the downsampled one

X = df_downsampled.drop(labels='true', axis=1)
y = df_downsampled.true


# Split in training/testing data but in a way that Tweets referring to the same article don't appear in both datasets
gs = GroupShuffleSplit(n_splits=2, test_size=.6, random_state=111019)
train_ix, test_ix = next(gs.split(X, y, groups=X.article_id))

# Clean dataset
X = clean_dataset(X)
# Do NLTK preprocessing
X['text'] = X.text.apply(nltk_preprocess)

X['created_at'] = pd.to_datetime(X['created_at']) # str to datetime
X['created_at'] = X.apply(lambda row: row['created_at'].hour, axis=1) # get hour from datetime

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

In [134]:
# Get features for TF-IDF

train_text = X_train['text']
train_labels = y_train

test_text = X_test['text']
test_labels = y_test

In [127]:
X.head()

,text,source,created_at,lang,description,username,verified,name,location
163133,私は 花火とほろよい で夏を楽しみたいシーンを選んでツイートすると沢尻さんが詠む夏の一句が届...,Twitter for Android,2018-07-23T03:45:49.000Z,ja,NaN,kirakirah0ppy,False,⭐きらきら⭐,NaN
204478,responsibility require one hundred percent pre...,Twittascope,2018-06-08T23:00:38.000Z,en,She got an ass that could swallow up a g-string,amy1thomas,False,lembit opik,NaN
203226,ironic reality starsarah palins cpac speech tr...,Twitter for Websites,2013-03-17T18:39:43.000Z,en,"M.D., pediatrics/adolescents. Aspiring comedia...",burnedoutdoc,False,J.J. Shaughnessy,"Los Angeles, CA, USA"
124057,ease living downtown austin paired luxury amen...,Facebook,2018-07-01T23:43:14.000Z,en,"austin tx realtor, wife, mom, wine lover, coff...",SusanAvant,False,Susan Avant,"Austin, TX"
202652,r v issue rt attackerman printout roll call am...,TweetDeck,2013-07-24T23:47:22.000Z,en,I write/edit @techdirt. I survived for 12 year...,mmasnick,True,Mike Masnick,NaN


# BERT

In [7]:
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=URn-DWJt5xhP
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder

In [9]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [22]:
print(padded)

[[  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 ...
 [  101 21907  2289 ...     0     0     0]
 [  101  4931  3071 ...     0     0     0]
 [  101  1030  3565 ...     0     0     0]]


In [13]:
padded_encoding = df['text'].apply(
    lambda src_text: tokenizer.encode_plus(
        src_text, add_special_tokens = True,
        truncation = True, padding = "max_length", 
        return_attention_mask = True, return_tensors = "pt")
)

In [14]:
#input_ids = torch.tensor(np.array(padded_encoding))

# print(padded_encoding.shape)

# #new_tensor = torch.tensor([200, padded_encoding])

new_tensor = padded_encoding[None, :]
#print(new_tensor)
new_tensor[0] = 200
#print(new_tensor.shape)

with torch.no_grad():
    last_hidden_states = model(new_tensor) # ERROR: Tries to get 255 GB of RAM

C:\Users\karli\AppData\Local\Temp\ipykernel_4036\3607074656.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  new_tensor = padded_encoding[None, :]


[[{'input_ids': tensor([[  101,  4911,  1024,  2034,  5088,  2136, 18806, 10528,  2058, 16916,
           24106,  1024, 16770,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
            2852,  2475,  3501, 13687,  2243,  2078,  2581,  2278,   102,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              

(1, 405102)


TypeError: 'int' object is not callable

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.5866666666666667

# Cleaning dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [81]:
df = clean_dataset(X)
df['text'] = df.text.apply(nltk_preprocess)
df.head()

,text,source,created_at,lang,description,username,verified,name,location
163133,私は 花火とほろよい で夏を楽しみたいシーンを選んでツイートすると沢尻さんが詠む夏の一句が届...,Twitter for Android,2018-07-23T03:45:49.000Z,ja,NaN,kirakirah0ppy,False,⭐きらきら⭐,NaN
204478,responsibility require one hundred percent pre...,Twittascope,2018-06-08T23:00:38.000Z,en,She got an ass that could swallow up a g-string,amy1thomas,False,lembit opik,NaN
203226,ironic reality starsarah palins cpac speech tr...,Twitter for Websites,2013-03-17T18:39:43.000Z,en,"M.D., pediatrics/adolescents. Aspiring comedia...",burnedoutdoc,False,J.J. Shaughnessy,"Los Angeles, CA, USA"
124057,ease living downtown austin paired luxury amen...,Facebook,2018-07-01T23:43:14.000Z,en,"austin tx realtor, wife, mom, wine lover, coff...",SusanAvant,False,Susan Avant,"Austin, TX"
202652,r v issue rt attackerman printout roll call am...,TweetDeck,2013-07-24T23:47:22.000Z,en,I write/edit @techdirt. I survived for 12 year...,mmasnick,True,Mike Masnick,NaN


In [142]:
import datetime
import pytz

def remove_date(row):
    #obj = datetime.strptime(row['created_at'], )
    return row['created_at'].timetz()

def get_hour(row):
    return row['created_at'].hour

def get_tz(row):
    time = row['created_at'].split('T')[1]
    time_split = re.split(r'[:|.]', time)
    return time_split[-1]

df = X.copy()
df['created_at'] = pd.to_datetime(df['created_at']) # str to datetime
# TODO: Put into section when importing dataset
df['created_at'] = df.apply(lambda row: row['created_at'].hour, axis=1)

#df.head()
df['created_at'].value_counts()

16    13005
17    12341
18    11815
15    11604
19    11578
20    11538
21    11022
14    10669
0     10016
13     9992
22     9912
23     9664
1      9481
2      9400
3      9272
12     9214
4      7765
11     7023
5      6899
10     5689
6      5580
7      5353
8      4682
9      4348
Name: created_at, dtype: int64

In [138]:
import re

def stringify_time(row):
    time = row['created_at'].split('T')[1]
    time_split = re.split(r'[:|.]', time)
    return " ".join(time_split)

df = X.copy()
df['created_at'] = df.apply(lambda row: stringify_time(row), axis=1)
df.head()

,text,source,created_at,lang,description,username,verified,name,location
163133,私は 花火とほろよい で夏を楽しみたいシーンを選んでツイートすると沢尻さんが詠む夏の一句が届...,Twitter for Android,03 45 49 000Z,ja,NaN,kirakirah0ppy,False,⭐きらきら⭐,NaN
204478,responsibility require one hundred percent pre...,Twittascope,23 00 38 000Z,en,She got an ass that could swallow up a g-string,amy1thomas,False,lembit opik,NaN
203226,ironic reality starsarah palins cpac speech tr...,Twitter for Websites,18 39 43 000Z,en,"M.D., pediatrics/adolescents. Aspiring comedia...",burnedoutdoc,False,J.J. Shaughnessy,"Los Angeles, CA, USA"
124057,ease living downtown austin paired luxury amen...,Facebook,23 43 14 000Z,en,"austin tx realtor, wife, mom, wine lover, coff...",SusanAvant,False,Susan Avant,"Austin, TX"
202652,r v issue rt attackerman printout roll call am...,TweetDeck,23 47 22 000Z,en,I write/edit @techdirt. I survived for 12 year...,mmasnick,True,Mike Masnick,NaN


In [122]:
import pytz

df = X
#print(df)
df['created_at'] = df.apply(lambda row: row['created_at'].astimezone('Europe/Riga'), axis=1)
df.head()

AttributeError: 'datetime.time' object has no attribute 'astimezone'

In [124]:
df.iloc[0]['created_at']

datetime.time(3, 45, 49, tzinfo=<UTC>)

In [ ]:
tweets_df2.to_csv("final_short.csv")

hello


## Multi-Layer Perceptron

In [12]:
# https://github.com/georgian-io/Multimodal-Toolkit/blob/master/multimodal_transformers/model/layer_utils.py

import math
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss


class MLP(nn.Module):
    """mlp can specify number of hidden layers and hidden layer channels"""

    def __init__(self, input_dim, output_dim, act='relu', num_hidden_lyr=2,
                 dropout_prob=0.5, return_layer_outs=False,
                 hidden_channels=None, bn=False):
        super().__init__()
        self.out_dim = output_dim
        self.dropout = nn.Dropout(dropout_prob)
        self.return_layer_outs = return_layer_outs
        if not hidden_channels:
            hidden_channels = [input_dim for _ in range(num_hidden_lyr)]
        elif len(hidden_channels) != num_hidden_lyr:
            raise ValueError(
                "number of hidden layers should be the same as the lengh of hidden_channels")
        self.layer_channels = [input_dim] + hidden_channels + [output_dim]
        self.act_name = act
        self.activation = create_act(act)
        self.layers = nn.ModuleList(list(
            map(self.weight_init, [nn.Linear(self.layer_channels[i], self.layer_channels[i + 1])
                                   for i in range(len(self.layer_channels) - 2)])))
        final_layer = nn.Linear(self.layer_channels[-2], self.layer_channels[-1])
        self.weight_init(final_layer,   activation='linear')
        self.layers.append(final_layer)

        self.bn = bn
        if self.bn:
            self.bn = nn.ModuleList([torch.nn.BatchNorm1d(dim) for dim in self.layer_channels[1:-1]])

    def weight_init(self, m, activation=None):
        if activation is None:
            activation = self.act_name
        torch.nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain(activation))
        return m

    def forward(self, x):
        """
        :param x: the input features
        :return: tuple containing output of MLP,
                and list of inputs and outputs at every layer
        """
        layer_inputs = [x]
        for i, layer in enumerate(self.layers):
            input = layer_inputs[-1]
            if layer == self.layers[-1]:
                layer_inputs.append(layer(input))
            else:
                if self.bn:
                    output = self.activation(self.bn[i](layer(input)))
                else:
                    output = self.activation(layer(input))
                layer_inputs.append(self.dropout(output))

        # model.store_layer_output(self, layer_inputs[-1])
        if self.return_layer_outs:
            return layer_inputs[-1], layer_inputs
        else:
            return layer_inputs[-1]

## Bert Concatenate Features

In [13]:
# https://www.youtube.com/watch?v=NbbsVcs42jE

from torch import nn
from transformers import BertForSequenceClassification

class BertConcatFeatures(BertForSequenceClassification):
    
    def __init__(self, config):
        super().__init__(config) # BERT setup
        
        self.num_labels = config.num_labels
        
        combined_feat_dim = config.text_feat_dim + \
                            config.cat_feat_dim + \
                            config.numerical_feat_dim
        
        self.num_bn = nn.BatchNorm1d(config.numerical_feat_dim)
        
        dims = []
        dim = combined_feat_dim
        while True:
            dim = dim // 4
            if dim <= self.num_labels:
                break 
            dim.append(int(dim))
        print('MLP layer sizes:')
        print('   Input:', combined_feat_dim)
        print('   Hidden:', dims)
        print('   Output:', self.num_labels)
        print('')
        
        self.mlp = MLP(combined_feat_dim,
                       self.num_labels,
                       num_hidden_layers=len(dims),
                       dropout_prob=0.1,
                       hidden_channels=dims,
                       bn=True)
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, inputs_embeds=None, labels=None,
                class_weights=None, output_attentions=None, output_hidden_states=None,
                cat_feats=None, numerical_feats=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states
                           )
        
        # outputs[0] - ALL output embeddings from BERT
        # outputs[1] - [CLS] token embedding, with additional "pooling" done
        cls = outputs[1]
        cls = self.dropout(cls)
        
        # Concatenate features
        
        numerical_feats = self.num_bn(numerical_feats) # batch-normalise numerical features
        combined_feats = torch.cat((cls, cat_feats, numerical_feats), dim=1)
        
        # Output classifier / Regression
        
        logits = self.mlp(combined_feats)
        
        if type(logits) is tuple: # regression
            logits, classifier_layer_outputs = logits[0], logits[1]
        else: # simple classifier
            classifier_layer_outputs = [combined_feats, logits]
            
        # Loss
        if labels is not None:
            # Regression
            if self.num_labels == 1:
                loss_fct = MSELoss()
                labels = labels.float()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            # Classification
            else:
                loss_fct = CrossEntropyLoss(weight=class_weights)
                labels = labels.long()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                
        else:
            loss = None
            
        results = {
            'loss': loss,
            'logits': logits,
            'classifier_layer_outputs': classifier_layer_outputs
        }
        
        return results

## Load Model

In [14]:
from transformers import BertConfig

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=1)
config.numerical_feat_dim = numerical_feats.size()[1]
config.cat_feat_dim = categorical_feats.size()[1]
config.text_feat_dim = config.hidden_size

model = BertConcatFeatures.from_pretrained(
    'bert-base-uncased',
    config = config
)

NameError: name 'numerical_feats' is not defined

In [ ]:
## Training

In [7]:
batch_size = 16
learning_rate = 3e-3
epochs = 4
print('Using maximum sequence length:', max_len)

NameError: name 'max_len' is not defined

In [83]:
print(len(data_df), len(y))

217862 217862


In [8]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

print(wrapper.fill(sen_w_feats[1]))

I created this Tweet at 2018-02-05T20:30:01.000Z o'clock in the en language and
in  from Twitter Web Client. My name is 17thReese, username is 17th_reese and I
am not verified. I would describe myself as . I wrote: NFL Lawyer, Who Claimed
Super Bowl Is ‘Rigged’, Found Dead #Topbuzz https://t.co/a9Lbddvxya


In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [10]:
from transformers import BertForSequenceClassification

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
max_len = 480

# # For every sentence...
# for sent in sen_w_feats:

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)

#     # Update the maximum sentence length.
#     max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len) # 474 from updating

Max sentence length:  480


In [12]:
# Larger batch sizes tend to be better, and we can fit this in memory.
batch_size = 8

# I used a smaller learning rate to combat over-fitting that I was seeing in the
# validation loss. I could probably try even smaller.
learning_rate = 1e-5

# Number of training epochs. 
epochs = 4

max_len = 480

In [13]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

print('Encoding all reviews in the dataset...')

# For every sentence...
for sent in sen_w_feats:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('DONE.')

Encoding all reviews in the dataset...
DONE.


C:\Users\karli\AppData\Local\Temp\ipykernel_324\2228480261.py:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  labels = torch.tensor(labels)


In [14]:
from torch.utils.data import TensorDataset

# Split the samples, and create TensorDatasets for each split. 
train_dataset = TensorDataset(input_ids[train_ix], attention_masks[train_ix], labels[train_ix])
val_dataset = TensorDataset(input_ids[val_ix], attention_masks[val_ix], labels[val_ix])
test_dataset = TensorDataset(input_ids[test_ix], attention_masks[test_ix], labels[test_ix])

print(len(train_dataset), len(val_dataset), len(test_dataset), len(train_dataset) + len(val_dataset) + len(test_dataset))

150164 9324 58374 217862


In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [37]:
from torch.optim import AdamW

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8 
                )

In [33]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples!)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [18]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Training Feature-Engineered BERT

In [20]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [43]:
print(b_input_ids[:3])
print(b_input_ids.shape)
print(b_labels.shape)
encoded_labels = torch.nn.functional.one_hot(labels.long()).float() #.astype(np.float32)
print(encoded_labels[:20])

# Split the samples, and create TensorDatasets for each split. 
train_dataset = TensorDataset(input_ids[train_ix], attention_masks[train_ix], encoded_labels[train_ix])
val_dataset = TensorDataset(input_ids[val_ix], attention_masks[val_ix], encoded_labels[val_ix])
test_dataset = TensorDataset(input_ids[test_ix], attention_masks[test_ix], encoded_labels[test_ix])

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

tensor([[ 101, 1045, 2580,  ...,    0,    0,    0],
        [ 101, 1045, 2580,  ...,    0,    0,    0],
        [ 101, 1045, 2580,  ...,    0,    0,    0]])
torch.Size([8, 480])
torch.Size([8, 2])
tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])


In [45]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 111019

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        #print(b_input_ids)
        #print(b_labels)
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update the learning rate.
        scheduler.step()

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()


    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  18,771.    Elapsed: 0:16:52.


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 9437184 bytes.

In [6]:
# Combine dataset classes
real_df = pd.read_csv("data/FakeNewsNet/dataset/pf_real.csv")
fake_df = pd.read_csv("data/FakeNewsNet/dataset/pf_fake.csv")
fake_df['true'] = False
real_df['true'] = True
df = pd.concat([fake_df, real_df])
df.reset_index()

#print(df.head())


# Fix imbalance
# https://elitedatascience.com/imbalanced-classes
# Separate majority and minority classes
df_majority = df[df.true==True]
df_minority = df[df.true==False]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

df_downsampled = df_downsampled.sample(frac=1, random_state=111019)

df_downsampled.reset_index(inplace=True)

# https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/#31-all-features-to-text
# This will hold all of the dataset samples, as strings.
sen_w_feats = []

# The labels for the samples.
labels = []

# First, reload the dataset to undo the transformations we applied for XGBoost.
data_df = df_downsampled.copy()

# Some of the reviews are missing either a "Title" or "Review Text", so we'll 
# replace the NaN values with empty string.
data_df = data_df.fillna("")

print('Combining features into strings...')

# text 	source 	created_at 	lang 
# description 	username 	verified 	name 	location

# For each of the samples...
#print(data_df.head())
print(len(data_df))
for index, row in data_df.iterrows():
    # Piece it together...    
    combined = ""
    
    combined += f"I created this Tweet at {row['created_at']} o'clock " + \
        f"in the {row['lang']} language and in {row['location']} " + \
        f"from {row['source']}. "
        
    verification_status = "verified" if row["verified"] else "not verified"
    combined += f"My name is {row['name']}, username is {row['username']} " + \
        f"and I am {verification_status}. "
    
    combined += f"I would describe myself as {row['description']}. "
    
    combined += f"I wrote: {row['text']} "
    
    # Add the combined text to the list.
    sen_w_feats.append(combined)

    # Also record the sample's label.
    #print(index)
    labels.append(data_df.iloc[index]['true'])

print('  DONE.')

print('Dataset contains {:,} samples.'.format(len(sen_w_feats)))



# Split train-test
# https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group

# Update df to the downsampled one

X = df_downsampled.drop(labels='true', axis=1)
y = df_downsampled.true

#print(y.head())

# NLTK preprocessing - not necessary with BERT
#X['text'] = X.text.apply(nltk_preprocess)

X['created_at'] = pd.to_datetime(X['created_at']) # str to datetime
X['created_at'] = X.apply(lambda row: row['created_at'].hour, axis=1) # get hour from datetime


# Split into train/val/test sets
# Technique gained from https://stackoverflow.com/questions/68152473/python-split-to-train-test-val-using-groupshufflesplit
# Start by splitting into train-val and test

gs_1 = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=111019)
train_val_ix, test_ix = next(gs_1.split(X, y, groups=X.article_id))

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

X_train_val = X.iloc[train_val_ix]
y_train_val = y.iloc[train_val_ix]

# Then split train-val into train and val (but reset index first)
X_train_val = X_train_val.reset_index(drop=True)
y_train_val = y_train_val.reset_index(drop=True)
gs_2 = GroupShuffleSplit(n_splits=1, train_size=0.9, random_state=111019)
train_ix, val_ix = next(gs_2.split(X_train_val, y_train_val, groups=X_train_val.article_id))

# Clean dataset, e.g., remove article_id column
X_train_val = clean_dataset(X_train_val)
X_test = clean_dataset(X_test)

X_train = X_train_val.iloc[train_ix]
y_train = y.iloc[train_ix]
X_val = X_train_val.iloc[val_ix]
y_val = y.iloc[val_ix]

Combining features into strings...
217862
  DONE.
Dataset contains 217,862 samples.


NameError: name 'clean_dataset' is not defined